<a href="https://colab.research.google.com/github/jefersonsr05/RoteiroMagico/blob/main/GerarRoteiro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Instalando packages necessários*

In [ ]:
!pip install -q -U google-generativeai
!pip install ipywidgets

*Imports de bicliotecas necessárias*

In [286]:
from google.colab import auth
from google.colab import files
from google.colab import userdata
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, widgets
from IPython.display import display
from pathlib import Path
from PIL import Image
import hashlib
import google.generativeai as genai
import io
from IPython.display import display
import markdown

Configurando a API *Key*

In [287]:
gemini_api_key=userdata.get('NEW_KEY')
genai.configure(api_key=gemini_api_key)

Definindo o padrão de Layout dos itens do formulario

In [288]:
form_item_layout = Layout(
  display='flex',
  flex='auto',
  justify_content='space-between'
)

Criando os widgets para configuração do modelo

In [289]:
temperature = widgets.FloatSlider(
  value=0.5,
  min=0,
  max=1.0,
  step=0.1,
  disabled=False,
  continuous_update=False,
  orientation='horizontal',
  readout=True,
  readout_format='.1f',
  layout=Layout(width='330px'),
)

harassment = widgets.Dropdown(options=['BLOCK_NONE', 'BLOCK_ONLY_HIGH', 'BLOCK_MEDIUM_AND_ABOVE', 'BLOCK_LOW_AND_ABOVE'])
hate = widgets.Dropdown(options=['BLOCK_NONE', 'BLOCK_ONLY_HIGH', 'BLOCK_MEDIUM_AND_ABOVE', 'BLOCK_LOW_AND_ABOVE'])
sexual = widgets.Dropdown(options=['BLOCK_NONE', 'BLOCK_ONLY_HIGH', 'BLOCK_MEDIUM_AND_ABOVE', 'BLOCK_LOW_AND_ABOVE'])
dangerous = widgets.Dropdown(options=['BLOCK_NONE', 'BLOCK_ONLY_HIGH', 'BLOCK_MEDIUM_AND_ABOVE', 'BLOCK_LOW_AND_ABOVE'])

box_configuration = [

    Box([Label(value='  Bora gerar um roteiro mágico baseado em uma imagem de aventura que você gosta!  ')], layout=Layout(
      display='flex',
      flex_flow='column',
      border='solid 2px',
    )),

    Box([Label(value='')]),
    Box([Label(value='Ajuste aqui as configurações do Gemini')]),
    Box([Label(value='')]),
    Box([Label(value='Temperature: ', layout=Layout(width='120px',)), temperature]),
    Box([Label(value='HARASSMENT: ', layout=Layout(width='120px',)), harassment]),
    Box([Label(value='      HATE: ', layout=Layout(width='120px',)), hate]),
    Box([Label(value='    SEXUAL: ', layout=Layout(width='120px',)), sexual]),
    Box([Label(value=' DANGEROUS: ', layout=Layout(width='120px',)), dangerous]),
    Box([Label(value='')]),
    Box([Label(value='')]),
]

Definindo o Layout para o Box da configuração

In [290]:
form_config = Box(box_configuration, layout=Layout(
      display='flex',
      flex_flow='column',
      border='solid 1px',
  ))

Criando os widgets para escolha do usuário

In [291]:
days_of_trip = widgets.IntSlider(
  value=3,
  min=1,
  max=30,
  layout=Layout(width='330px'),
)

quantity = widgets.IntSlider(
  value=3,
  min=1,
  max=5,
  layout=Layout(width='330px'),
)

file_upload = widgets.FileUpload(
    accept='image/*',
    description='Selecione uma foto ... ',
    button_style='success',
    layout=Layout(width='425px'),
    multiple=False
)

output = widgets.Output()

def salvar_imagem_selecionada(file_upload, name):
    uploaded_filenames = list(file_upload.value.keys())
    if len(uploaded_filenames) == 1:
        uploaded_filename = uploaded_filenames[0]
        uploaded_file = file_upload.value[uploaded_filename]['content']

        # Carregar imagem
        imagem = Image.open(io.BytesIO(uploaded_file))

        # Salvar imagem
        nome_do_arquivo = name  # Nome do arquivo que deseja salvar
        imagem.save(nome_do_arquivo)

        print(f"Imagem salva como '{nome_do_arquivo}'")
    else:
        print("Por favor, selecione uma imagem para salvar.")

# função para mesmo que o usuario escolha mais de uma foto o sistema pega sempre a ultima
def on_upload_change(change):
    with output:
        output.clear_output()
        uploaded_filenames = list(file_upload.value.keys())
        if len(uploaded_filenames) > 1:
            print("Por favor, selecione apenas uma imagem.")
            file_upload.value.clear()
        elif len(uploaded_filenames) == 1:
            print("Imagem selecionada: ", uploaded_filenames[0])
            # Chamando a função para salvar a imagem selecionada
            salvar_imagem_selecionada(file_upload, uploaded_filenames[0])
        else:
            print("Nenhuma imagem selecionada.")

# Observer para cada vez que o usuario escolher uma foto o sistema substitui a anterior e salva no COLAB a ultima
file_upload.observe(on_upload_change, names='_counter')

# Design do visual para mostrar mais amigavel ao usuario final
box_input_user = [
    Box([Label(value='Agora vamos programar sua viagem!')]),
    Box([Label(value='')]),
    Box([Label(value='Dias de viagem: ', layout=Layout(width='120px',)), days_of_trip]),
    Box([Label(value='Quantas de dicas:', layout=Layout(width='120px',)), quantity]),
    Box([Label(value='')]),
    Box([file_upload]),
    Box([output]),
]

Definindo a função responsável por sugerir as viagens via IA

In [292]:
def suggest_trips():
  trip_suggestions = []

  for uploaded_file in file_upload.value:
    print(uploaded_file)
    name = uploaded_file

  image_url = name

  # Fazendo a consulta ao Gemini
  generation_config = {
    "candidate_count": 1,
    "temperature":  temperature.value,
  }

  safety_settings = {
    "HARASSMENT": harassment.value,
    "HATE": hate.value,
    "SEXUAL": sexual.value,
    "DANGEROUS": dangerous.value,
  }

  model = genai.GenerativeModel(
    model_name="gemini-1.5-pro-latest",
    generation_config = generation_config,
    safety_settings=safety_settings
    )

  prompt_parts = [
    genai.upload_file(f"/content/{image_url}"),
    f"Com base na imagem anexada, me de {quantity.value} dicas de viagens semelhantes para um roteiro de {days_of_trip.value} dias. Sugira alguns destinos, link de imagem do google imagens e descrição de pontos interessantes a serem visitados e tambem qual a melhor epoca de visitação",
  ]

  response = model.generate_content(prompt_parts)

  return response.text

Layout dos botões de ação do usuario

In [293]:
form_input_user = Box(box_input_user, layout=Layout(
      display='flex',
      flex_flow='column',
      border='solid 1px',
  ))


Botão de ação para geração de roteiro e chamada da função para gerar sugestão de viagens

In [298]:
get_trip = widgets.Button(
    value=False,
    description='Gerar Roteiro ...',
    disabled=False,
    button_style='',
    icon='check',
    layout=Layout(width='425px'),
)

def on_click(change):
    # Imprimindo sugestões de viagens e mostrando de forma amigavel via um component que interpreta o formato markdown
    print("Sugestões de viagens com base na sua foto:  AGUARDE...")
    markdown_text = suggest_trips()
    html_text = markdown.markdown(markdown_text)
    html_widget = widgets.HTML(html_text)
    html_widget.layout.width = '800px'  # Adjust widget width as needed
    display(html_widget)

get_trip.on_click(on_click)

Box do botão gerar viagem

In [299]:
box_button_get = [
    Box([get_trip]),
]

Layout do box para botões de ação do usuario

In [300]:
form_get = Box(box_button_get, layout=Layout(
      display='flex',
      flex_flow='column',
      border='solid 1px',
  ))

Criação do display com todos os widgets criados anteriormente

In [301]:
file_upload.close
display(form_config)
display(form_input_user)
display(form_get)

Box(children=(Box(children=(Label(value='  Bora gerar um roteiro mágico baseado em uma imagem de aventura que …

Box(children=(Box(children=(Label(value='Agora vamos programar sua viagem!'),)), Box(children=(Label(value='')…

Box(children=(Box(children=(Button(description='Gerar Roteiro ...', icon='check', layout=Layout(width='425px')…

Sugestões de viagens com base na sua foto:  AGUARDE...
cachoeira.jpg


HTML(value="<h2>3 Dicas de Viagens Semelhantes à Imagem:</h2>\n<p>A imagem mostra uma mulher em um caiaque apr…